In [2]:
%cd ..

C:\Users\chibok


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt # module for plotting
import seaborn as sns
import sklearn as sklearn
from sklearn import linear_model, cross_validation, metrics, svm, ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.cross_validation  import train_test_split, cross_val_score, ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy.stats import sem
import statsmodels.api as sm
import itertools
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

import matplotlib.pyplot as plt # module for plotting 
from matplotlib import interactive, font_manager
from matplotlib import rcParams
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.colors import ListedColormap
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843),
                (0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843)]

cmap_set1 = ListedColormap(['#4daf4a', '#377eb8', '#e41a1c' ])
dark2_cmap=ListedColormap(dark2_colors)

In [4]:
######################################
######################################
# Matplotlib config code taken from CS109 2013 
######################################
def set_mpl_params():
    rcParams['figure.figsize'] = (10, 6)
    rcParams['figure.dpi'] = 150
    rcParams['axes.color_cycle'] = dark2_colors
    rcParams['lines.linewidth'] = 2
    rcParams['axes.facecolor'] = 'white'
    rcParams['font.size'] = 14
    rcParams['patch.edgecolor'] = 'white'
    rcParams['patch.facecolor'] = dark2_colors[0]
    rcParams['font.family'] = 'StixGeneral'

set_mpl_params()

######################################
######################################
# Remove border function taken from CS109 2013 
######################################
def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecessary plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()
        
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

In [5]:
# plot confustion matrix function taken from class notes
def show_confusion_matrix(cm):
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.binary)
    plt.title('Confusion matrix')
    plt.set_cmap('Blues')
    plt.colorbar()

    target_names = ['Not Delayed', 'Delayed']

    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=60)
    plt.yticks(tick_marks, target_names)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # Convenience function to adjust plot parameters for a clear layout.
    plt.show()

# 추가 되는 자료 전처리하기

In [6]:
def predict_X( wind =0, wind2 = 8, dist = 1000 , temp = 10, rain = 0 , the_number_of_delay = 2, company= u'대한항공', month = 3, day = 10, hour = u"8", weekday = 5):
    df_data = pd.read_csv('df_data.csv', encoding = 'cp949')
    df_raw_X = pd.DataFrame(df_data, columns=["wind", "wind2","dist","temp", "rain", "the number of delay", "company", u"month", u"day", u"hour", "weekday"])
    raw_X=df_raw_X.values
    predict_values = [0, 8, 1000, 10, 0, 2, u'대한항공', 2, 13, u'8H', 5]
    raw_X = np.vstack((raw_X, predict_values))
    
    df_X = pd.DataFrame(raw_X, columns=["wind", "wind2","dist","temp", "rain", "the number of delay", "company", u"month", u"day", u"hour", "weekday"])
    df_X.to_csv("predict_X.csv", encoding= 'cp949')
    
    
    df_X = pd.read_csv('predict_X.csv', encoding='cp949')
    
    df_X = pd.DataFrame(df_X, columns=["wind", "wind2","dist","temp", "rain", "the number of delay", "company", u"month", u"day", u"hour", "weekday"])
    
    df_X.loc[:,u"company"] = LabelEncoder().fit_transform(df_X[u"company"])
    dfX2 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"company"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["koreanair","asianair","busanair","estarair","jejuair","jinair","taway"], index=df_X.index)
    df_X = pd.concat([df_X, dfX2], axis=1)
    del(df_X[u"company"])
    
    df_X.loc[:,u"month"] = LabelEncoder().fit_transform(df_X[u"month"])
    dfX6 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"month"].as_matrix()[:,np.newaxis]).toarray(), 
                        columns=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], index=df_X.index)
    df_X = pd.concat([df_X, dfX6], axis=1)
    del(df_X[u"month"])
    
    dfX4 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"day"].as_matrix()[:,np.newaxis]).toarray(), 
                        columns=["day1","day2","day3","day4","day5","day6","day7","day8","day9","day10","day11","day12","day13","day14","day15","day16","day17","day18","day19","day20","day21","day22","day23","day24","day25","day26","day27","day28","day29","day30","day31"], index=df_X.index)
    df_X = pd.concat([df_X, dfX4], axis=1)
    del(df_X[u"day"])
    
    df_X.loc[:,"hour"] = LabelEncoder().fit_transform(df_X["hour"])
    dfX3 = pd.DataFrame(OneHotEncoder().fit_transform(df_X["hour"].as_matrix()[:,np.newaxis]).toarray(), 
                        columns=["0H", "1H", "4H", "5H", "6H", "7H", "8H", "9H", "10H", "11H", "12H", "13H", "14H", "15H", "16H", "17H", "18H", "19H", "20H", "21H", "22H", "23H"], index=df_X.index)
    df_X = pd.concat([df_X, dfX3], axis=1)
    
    del(df_X[u"hour"])
    
    
    
    dfX5 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"weekday"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["sun","mon","tue","wen","thus","fri", "sat"], index=df_X.index)
    df_X = pd.concat([df_X, dfX5], axis=1)
    del(df_X[u"weekday"])
    predict_X=df_X.values
    predict_X[-1]
    return predict_X[-1]

In [7]:
 predictX =predict_X(0, 8, 1000, 10, 0, 2, u'대한항공', 3, 13, u'8H', 5)

# 알고리즘 training시키기(기존 자료로)

In [13]:
def training():
    df_raw_data_def = pd.read_csv("df_data.csv", encoding = 'cp949')
    df_X = pd.DataFrame(df_raw_data_def, columns=["wind", "wind2","dist","temp", "rain", "the number of delay", "company", "month", "day", "hour", "weekday"])
    df_y = pd.DataFrame(df_raw_data_def, columns=["status"])
    df_X.loc[:,u"company"] = LabelEncoder().fit_transform(df_X[u"company"])
    dfX2 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"company"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["koreanair","asianair","busanair","estarair","jejuair","jinair","taway"], index=df_X.index)
    df_X = pd.concat([df_X, dfX2], axis=1)
    del(df_X[u"company"])
    
    df_X.loc[:,u"month"] = LabelEncoder().fit_transform(df_X[u"month"])
    dfX6 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"month"].as_matrix()[:,np.newaxis]).toarray(), 
                        columns=[1,2,3,4,5,6,7,8,9,10,11,12], index=df_X.index)
    df_X = pd.concat([df_X, dfX6], axis=1)
    del(df_X[u"month"])
    
    df_X.loc[:,"hour"] = LabelEncoder().fit_transform(df_X["hour"])
    dfX3 = pd.DataFrame(OneHotEncoder().fit_transform(df_X["hour"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["0H", "1H","4H", "5H", "6H", "7H", "8H", "9H", "10H", "11H", "12H", "13H", "14H", "15H", "16H", "17H", "18H", "19H", "20H", "21H", "22H", "23H"], index=df_X.index)
    df_X = pd.concat([df_X, dfX3], axis=1)
    del(df_X["hour"])
    
    dfX4 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"day"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["day1","day2","day3","day4","day5","day6","day7","day8","day9","day10","day11","day12","day13","day14","day15","day16","day17","day18","day19","day20","day21","day22","day23","day24","day25","day26","day27","day28","day29","day30","day31"], index=df_X.index)
    df_X = pd.concat([df_X, dfX4], axis=1)
    del(df_X[u"day"])
    
    dfX5 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"weekday"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["sun","mon","tue","wen","thus","fri", "sat"], index=df_X.index)
    df_X = pd.concat([df_X, dfX5], axis=1)
    del(df_X[u"weekday"])
    
    from sklearn.cross_validation import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(df_X,df_y, test_size=0.15, random_state=20)
    
    X = X_train.values
    y = y_train.values
    testX = X_test
    testy = y_test
    
    clf_logic = LogisticRegression(C=1, penalty='l2', class_weight = 'auto')
    clf_rf = RandomForestClassifier(n_estimators=1, n_jobs=-1 ,class_weight='auto')
   
    
    eclf = VotingClassifier(estimators=[('lr', clf_logic), ('rf', clf_rf)],
                        voting='soft',
                        weights=[10, 2])
    eclf_model = eclf.fit(X, y)
    return eclf_model
    

In [14]:
eclf_model=training()

C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\preprocessing\label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\ensemble\voting_classifier.py:126: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  fitted_clf = clone(clf).fit(X, self.le_.transform(y))
C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change th

# "%" 나오게 정의

In [11]:
def delay_rate(X = predictX):
    pred = eclf_model.predict_proba(X)
    print ('Your flight delay_rate is', float(pred[0][1]), '%')
    return

# 예측하기


In [19]:
delay_rate(predictX)

Your flight delay_rate is 0.508650930858 %


C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [17]:
    df_raw_data_def = pd.read_csv("df_data.csv", encoding = 'cp949')
    df_X = pd.DataFrame(df_raw_data_def, columns=["wind", "wind2","dist","temp", "rain", "the number of delay", "company", "month", "day", "hour", "weekday"])
    df_y = pd.DataFrame(df_raw_data_def, columns=["status"])
    df_X.loc[:,u"company"] = LabelEncoder().fit_transform(df_X[u"company"])
    dfX2 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"company"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["koreanair","asianair","busanair","estarair","jejuair","jinair","taway"], index=df_X.index)
    df_X = pd.concat([df_X, dfX2], axis=1)
    del(df_X[u"company"])
    
    df_X.loc[:,u"month"] = LabelEncoder().fit_transform(df_X[u"month"])
    dfX6 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"month"].as_matrix()[:,np.newaxis]).toarray(), 
                        columns=[1,2,3,4,5,6,7,8,9,10,11,12], index=df_X.index)
    df_X = pd.concat([df_X, dfX6], axis=1)
    del(df_X[u"month"])
    
    df_X.loc[:,"hour"] = LabelEncoder().fit_transform(df_X["hour"])
    dfX3 = pd.DataFrame(OneHotEncoder().fit_transform(df_X["hour"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["0H", "1H","4H", "5H", "6H", "7H", "8H", "9H", "10H", "11H", "12H", "13H", "14H", "15H", "16H", "17H", "18H", "19H", "20H", "21H", "22H", "23H"], index=df_X.index)
    df_X = pd.concat([df_X, dfX3], axis=1)
    del(df_X["hour"])
    
    dfX4 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"day"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["day1","day2","day3","day4","day5","day6","day7","day8","day9","day10","day11","day12","day13","day14","day15","day16","day17","day18","day19","day20","day21","day22","day23","day24","day25","day26","day27","day28","day29","day30","day31"], index=df_X.index)
    df_X = pd.concat([df_X, dfX4], axis=1)
    del(df_X[u"day"])
    
    dfX5 = pd.DataFrame(OneHotEncoder().fit_transform(df_X[u"weekday"].as_matrix()[:,np.newaxis]).toarray(), 
                    columns=["sun","mon","tue","wen","thus","fri", "sat"], index=df_X.index)
    df_X = pd.concat([df_X, dfX5], axis=1)
    del(df_X[u"weekday"])
    
    from sklearn.cross_validation import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(df_X,df_y, test_size=0.15, random_state=20)
    
    X = X_train.values
    y = y_train.values
    testX = X_test
    testy = y_test
    
    clf_logic = LogisticRegression(C=1, penalty='l2', class_weight = 'auto')
    clf_rf = RandomForestClassifier(n_estimators=1, n_jobs=-1 ,class_weight='auto')
    clf_svc = SVC(kernel='rbf', C=100, gamma=0.001, class_weight ='auto')
    
    eclf = VotingClassifier(estimators=[('lr', clf_logic), ('rf', clf_rf)],
                        voting='soft',
                        weights=[10, 2])
    eclf_model = eclf.fit(X, y)
    eclf_pred = eclf_model.predict(testX)
    from sklearn import metrics
    print(metrics.classification_report(testy, eclf_pred))

C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\preprocessing\label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\ensemble\voting_classifier.py:126: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  fitted_clf = clone(clf).fit(X, self.le_.transform(y))


             precision    recall  f1-score   support

          0       0.92      0.80      0.85      6092
          1       0.36      0.61      0.45      1126

avg / total       0.83      0.77      0.79      7218



In [18]:
pred = eclf_model.predict_proba(predictX)
pred

C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\chibok\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.49134907,  0.50865093]])